In [1]:
# import sys
# sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/PyTorch27/lib/python2.7/site-packages')
# sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/PyTorch27/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg')
# sys.path.insert(0,'/vol/medic02/users/ag6516/miniconda/envs/PyTorch27/lib/python2.7/site-packages/IPython/extensions')
# sys.path
import csv
import pandas as pd
from read_data import read_csv_into_df

In [31]:
fields = ['Accession', 'Clinical history', 'Comment', 'Report text']
# dir = ('/vol/medic02/users/ag6516/x_ray_fracture_localisation/')
dir = ('/Users/Aydan/PhD/x_ray_fracture_localisation/')

with open(dir + 'data/Reports/XKNEB_Jan_Dec_2015_anon.csv', 'rU') as csvfile:
    data1 = csv.reader(csvfile, delimiter=',')
    headers1 = data1.next()

with open(dir + 'data/Reports/XKNEB_Jan_Dec_2016_anon.csv', 'rU') as csvfile:
    data2 = csv.reader(csvfile, delimiter=',')
    headers2 = data2.next()

# with open(dir + 'data/Reports/XKNEB_Jan_Jul_2017_anon.csv', 'rU') as csvfile:
#     data3 = csv.reader(csvfile, delimiter=',')
#     headers3 = data3.next()

cols1 = [headers1.index(item) for item in fields]
cols2 = [headers2.index(item) for item in fields]
# cols3 = [headers3.index(item) for item in fields]

In [32]:
df1 = read_csv_into_df(dir + 'data/Reports/XKNEB_Jan_Dec_2015_anon.csv', fields, cols1)
df2 = read_csv_into_df(dir + 'data/Reports/XKNEB_Jan_Dec_2016_anon.csv', fields, cols2)

In [33]:
df = pd.concat([df1,df2])
print df.count()

Accession           3576
Clinical history    2232
Comment             2402
Report text         3576
dtype: int64


In [34]:
# Check for missing information
df_nans = df[df.isnull().any(axis=1)]
df_nans.to_csv('missing_info.csv', sep=',')

In [35]:
df.to_csv('cleaned_reports.csv', sep=',')

In [99]:
# Basic filtering and stats
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import random
random.seed(123)
np.random.seed(123)

reports = []
labels = []
words = []
for i, row in df.iterrows():
    report = row['Report text'].decode('utf-8').lower()
    
    # Replace unnecessary punctuation
    
    for char in ['~', '"']:
        report = report.replace(char, '')
        
    for char in ['!', '?', ';', ':', '.']:
        report = report.replace(char, ' . ')
    
    for char in ['(', ')', ',']:
        report = report.replace(char, ' , ')
        
    # Tokenize
    report_tok = word_tokenize(report)
    
    # Remove stopwords
    filtered_report = [word for word in report_tok if word not in stopwords.words('english')]
    
    reports.append(filtered_report)
    labels.append(row['Accession'])
    [words.append(word) for word in filtered_report]

print 'Vocab length: ', len(sorted(set(words)))
print 'Avg no. of appearances: ', len(words)/len(sorted(set(words)))
fdist_all = nltk.FreqDist(words)
freqs = [freq for _, freq in fdist_all.most_common()]
print 'STD: ', np.std(freqs)
#print fdist_all.hapaxes()

Vocab length:  2158
Avg no. of appearances:  41
STD:  349.019096469


In [100]:
# train doc2vec model
import gensim
from gensim.models.doc2vec import LabeledSentence

class DocIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield LabeledSentence(words=doc, tags=[self.labels_list[idx]])
            
it = DocIterator(reports, labels)

In [101]:
# Train model
model = gensim.models.Doc2Vec(size=300, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(it)
#training of model
for epoch in range(10):
    print 'iteration ' +str(epoch+1)
    model.train(it, total_examples=model.corpus_count, epochs=model.iter)
    model.alpha -= 0.002
    model.min_alpha = model.alpha
    model.train(it, total_examples=model.corpus_count, epochs=model.iter)
#saving the created model
model.save('doc2vec.model')

iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10


In [102]:
# Test model
label_sample = random.choice(labels)
report_sample = df.loc[df['Accession'] == label_sample]['Report text']
print report_sample.to_string()

most_similar = model.docvecs.most_similar(label_sample)
print "Most similar documents:"

for label, _ in most_similar:
    print df.loc[df['Accession'] == label]['Report text']

349    XR Knee Both : \n\nLeft knee: There is severe ...
Most similar documents:
2538    XR Knee Both : Both knees have severe patellof...
Name: Report text, dtype: object
3176    Left knee: TKR in situ, good alignment. No adv...
Name: Report text, dtype: object
2691    XR Knee Both : In the right knee there is seve...
Name: Report text, dtype: object
1208    XR Knee Both : \nBilateral OA changes in both ...
Name: Report text, dtype: object
208    XR Knee Both : minor OA changes both patello f...
Name: Report text, dtype: object
3149    Left knee: There is severe OA change affecting...
Name: Report text, dtype: object
76    XR Knee Both : Early OA changes Patello femora...
Name: Report text, dtype: object
3121    XR Knee Both : There is moderate OA change to ...
Name: Report text, dtype: object
2365    XR Knee Both : No significant bone or joint ab...
Name: Report text, dtype: object
2853    XR Knee Both : \n\nLeft knee: There is moderat...
Name: Report text, dtype: object
